In [1]:
# First, install required packages (run this cell once)
!pip install SpeechRecognition pyttsx3
import speech_recognition as sr
import pyttsx3
import time
import csv
from datetime import datetime
from IPython.display import display, clear_output
import ipywidgets as widgets
class SpeechDataEntrySystem:
    def __init__(self):
        # Initialize speech recognition
        self.recognizer = sr.Recognizer()
        self.microphone = sr.Microphone()
        
        # Initialize text-to-speech engine
        self.engine = pyttsx3.init()
        self.engine.setProperty('rate', 150)  # Speed of speech
        
        # Data storage
        self.data = []
        self.fields = ['name', 'age', 'email', 'timestamp']
        
        # Jupyter widgets
        self.output = widgets.Output()
        self.start_button = widgets.Button(description="Start Data Entry")
        self.start_button.on_click(self.start_collection)
        
        display(self.start_button, self.output)
        
    def speak(self, text):
        """Convert text to speech and speak it"""
        with self.output:
            clear_output(wait=True)
            print(f"System: {text}")
        self.engine.say(text)
        self.engine.runAndWait()
        
    def listen(self):
        """Listen to user speech and convert to text"""
        with self.output:
            print("Listening...")
            
        with self.microphone as source:
            self.recognizer.adjust_for_ambient_noise(source)
            audio = self.recognizer.listen(source, timeout=5)
            
        try:
            text = self.recognizer.recognize_google(audio)
            with self.output:
                clear_output(wait=True)
                print(f"User: {text}")
            return text.lower()
        except sr.UnknownValueError:
            self.speak("Sorry, I didn't understand that. Please try again.")
            return None
        except sr.RequestError:
            self.speak("Sorry, my speech service is down. Please try again later.")
            return None
        except sr.WaitTimeoutError:
            self.speak("I didn't hear anything. Please try again.")
            return None
    
    def validate_email(self, email):
        """Basic email validation"""
        return '@' in email and '.' in email.split('@')[-1]
    
    def validate_age(self, age_text):
        """Age validation"""
        try:
            age = int(age_text)
            return 0 < age < 120
        except ValueError:
            return False
    
    def start_collection(self, button):
        """Start data collection when button is clicked"""
        self.collect_data()
    
    def collect_data(self):
        """Main method to collect data through voice"""
        self.speak("Welcome to the speech based data entry system.")
        
        # Get name
        self.speak("Please say your full name.")
        name = None
        while not name:
            name = self.listen()
        
        # Get age
        age = None
        while not age:
            self.speak("Please say your age.")
            age_text = self.listen()
            if age_text and self.validate_age(age_text):
                age = age_text
            else:
                self.speak("That doesn't sound like a valid age. Please say a number between 1 and 120.")
        
        # Get email
        email = None
        while not email:
            self.speak("Please say your email address.")
            email_text = self.listen()
            if email_text and self.validate_email(email_text):
                email = email_text.replace(' at ', '@').replace(' dot ', '.')
            else:
                self.speak("That doesn't sound like a valid email. Please try again.")
        
        # Create record
        record = {
            'name': name,
            'age': age,
            'email': email,
            'timestamp': datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        }
        
        self.data.append(record)
        self.save_to_csv(record)
        
        self.speak(f"Thank you {name}. Your information has been saved.")
        self.speak(f"Let me confirm your details. You are {age} years old and your email is {email}.")
        self.speak("Data entry complete. Goodbye!")
    
    def save_to_csv(self, record):
        """Save data to CSV file"""
        file_exists = False
        try:
            with open('data_entries.csv', 'r') as f:
                file_exists = True
        except FileNotFoundError:
            pass
            
        with open('data_entries.csv', 'a', newline='') as f:
            writer = csv.DictWriter(f, fieldnames=self.fields)
            if not file_exists:
                writer.writeheader()
            writer.writerow(record)
            # Create and run the system
system = SpeechDataEntrySystem()

Button(description='Start Data Entry', style=ButtonStyle())

Output()